In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
from itertools import combinations
import pandas as pd
%pip install gurobipy
import openpyxl
%pip install xlsxwriter
import xlsxwriter
from pathlib import Path

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 3.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 4.2 MB/s 


In [2]:
def read_data(name):
  X = np.loadtxt(open("/content/drive/My Drive/Bernice Research/June 17/%s.csv" % name, "rb"), delimiter=",", skiprows=1)
  y = X[:, -1] 
  X = X[:, 0:-1]
  X = np.c_[np.ones(X.shape[0]),X]
  return X, y

In [3]:
#LAD-LASSO LP for defg
def LAD_LASSO(X, y, lbd=0):
  import gurobipy as gp
  from gurobipy import GRB
  from gurobipy import quicksum

  n, p = X.shape

  m = gp.Model()

  m.Params.LogToConsole = 0

  rpls = m.addMVar((n,), vtype=GRB.CONTINUOUS)
  rmns = m.addMVar((n,), vtype=GRB.CONTINUOUS)
  bpls = m.addMVar((p,), vtype=GRB.CONTINUOUS)
  bmns = m.addMVar((p,), vtype=GRB.CONTINUOUS)

  m.update()

  m.addConstr(rpls - rmns + X @ bpls - X @ bmns  == y)
        
  m.setObjective(quicksum(rpls[i] for i in range(n)) + quicksum(rmns[i] for i in range(n)) + lbd*(quicksum(bpls[j] for j in range(1,p)) + quicksum(bmns[j] for j in range(1,p))), GRB.MINIMIZE)

  m.optimize()

  return round(m.objval, 6), [round(num, 6) for num in list(bpls.X-bmns.X)]
  # print(rpls.X-rmns.X)
  # print(bpls.X)
  # print(bmns.X)
  # print(rpls.X)
  # print(rmns.X)

In [4]:
#LAD-LASSO LP for defg
def LAD_LASSO_C(X, y, lbd=0):
  import gurobipy as gp
  from gurobipy import GRB
  from gurobipy import quicksum

  n, p = X.shape

  m = gp.Model()

  m.Params.LogToConsole = 0

  rpls = m.addMVar((n,), vtype=GRB.CONTINUOUS)
  rmns = m.addMVar((n,), vtype=GRB.CONTINUOUS)
  bpls = m.addMVar((p,), vtype=GRB.CONTINUOUS)
  bmns = m.addMVar((p,), vtype=GRB.CONTINUOUS)

  m.update()

  m.addConstr(rpls - rmns + X @ bpls - X @ bmns  == y)
        
  m.setObjective(quicksum(rpls[i] for i in range(n)) + quicksum(rmns[i] for i in range(n)) + lbd*(quicksum(bpls[j] for j in range(p)) + quicksum(bmns[j] for j in range(p))), GRB.MINIMIZE)

  m.optimize()

  return round(m.objval, 6), [round(num, 6) for num in list(bpls.X-bmns.X)]
  # return "%.6f" % m.objval, ["%.6f" % num for num in list(bpls.X-bmns.X)]
  # print(rpls.X-rmns.X)
  # print(bpls.X)
  # print(bmns.X)
  # print(rpls.X)
  # print(rmns.X)

In [5]:
def mean_center_scale(name):
  #Center and Scale dataset by mean
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  sd = np.zeros(l)
  for i in range(l):
    c[i] = np.mean(X[:,i])
    sd[i] = np.std(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = (X[j,k]-c[k])/sd[k]
  X = X[:,1:]
  my = np.mean(y)
  for j in range(len(X)):
    y[j] = y[j] - my
  n, p = X.shape
  return X,y

In [6]:
def mean_center(name):
  #Center but not Scale dataset by mean
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  for i in range(l):
    c[i] = np.mean(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]-c[k]
  X = X[:,1:]
  my = np.mean(y)
  for j in range(len(X)):
    y[j] = y[j] - my
  n, p = X.shape
  return X,y

In [7]:
def sd_scale(name):
  #Scale but not Center dataset by mean
  X,y = read_data(name)
  l = len(X[0])
  sd = np.zeros(l)
  for i in range(l):
    sd[i] = np.std(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]/sd[k]
  X = X[:,1:]
  X = np.c_[np.ones(X.shape[0]),X]
  n, p = X.shape
  return X,y

In [8]:
def sd_scale_y(name):
  #Scale but not Center dataset by mean
  X,y = read_data(name)
  l = len(X[0])
  sd = np.zeros(l)
  for i in range(l):
    sd[i] = np.std(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]/sd[k]
  X = X[:,1:]
  X = np.c_[np.ones(X.shape[0]),X]
  sdy = np.std(y)
  for j in range(len(X)):
    y[j] = y[j]/sdy
  n, p = X.shape
  return X,y

In [9]:
def sd_scale_y(name):
  #Scale but not Center dataset by mean
  X,y = read_data(name)
  l = len(X[0])
  sd = np.zeros(l)
  for i in range(l):
    sd[i] = np.std(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]/sd[k]
  X = X[:,1:]
  X = np.c_[np.ones(X.shape[0]),X]
  sdy = np.std(y)
  for j in range(len(X)):
    y[j] = y[j]/sdy
  n, p = X.shape
  return X,y

In [10]:
def sd_y(name):
  X,y = read_data(name)
  sdy = np.std(y)
  return sdy

In [11]:
sd_y('wood')

0.04609555293084139

In [12]:
def median_center_scale(name):
  #Center and Scale dataset by median
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  sd = np.zeros(l)
  for i in range(l):
    c[i] = np.median(X[:,i])
    sd[i] = np.mean(np.absolute(X[:,i] - np.median(X[:,i])))
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = (X[j,k]-c[k])/sd[k]
  X = X[:,1:]
  my = np.median(y)
  for j in range(len(X)):
    y[j] = y[j] - my
  n, p = X.shape
  return X,y

In [13]:
def median_center(name):
  #Center but not Scale dataset by median
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  sd = np.zeros(l)
  for i in range(l):
    c[i] = np.median(X[:,i])
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]-c[k]
  X = X[:,1:]
  my = np.median(y)
  for j in range(len(X)):
    y[j] = y[j] - my
  n, p = X.shape
  return X,y

In [14]:
def mad_scale(name):
  #Scale but not Center dataset by median
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  sd = np.zeros(l)
  for i in range(l):
    sd[i] = np.mean(np.absolute(X[:,i] - np.median(X[:,i])))
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]/sd[k]
  X = X[:,1:]
  X = np.c_[np.ones(X.shape[0]),X]
  n, p = X.shape
  return X,y

In [15]:
def mad_scale_y(name):
  #Scale but not Center dataset by median
  X,y = read_data(name)
  l = len(X[0])
  c = np.zeros(l)
  sd = np.zeros(l)
  for i in range(l):
    sd[i] = np.mean(np.absolute(X[:,i] - np.median(X[:,i])))
  for j in range(len(X)):
    for k in range(l):
      X[j,k] = X[j,k]/sd[k]
  X = X[:,1:]
  X = np.c_[np.ones(X.shape[0]),X]
  sdy = np.mean(np.absolute(y - np.median(y)))
  for j in range(len(X)):
    y[j] = y[j] /sdy
  n, p = X.shape
  return X,y

In [16]:
def mad_y(name):
  X,y = read_data(name)
  sdy = np.mean(np.absolute(y - np.median(y)))
  return sdy

In [17]:
mad_y('wood')

0.036300000000000006

In [18]:
def breakdown(X):
  n,p = X.shape
  c = list(combinations(list(range(0,n)), p-1))
  absxtimexi_list = []
  count = 0
  for i in range(len(list(c))):
    Xb = np.take(X, list(c[i]), 0)
    r = np.array([np.r_[1,np.zeros(p-1)]])
    xx = np.r_[r,Xb]
    v = np.matrix.transpose(np.zeros((1,p-1)))
    v = np.r_[np.array([np.array([1])]),v]
    try:
      xi = np.matmul(np.linalg.inv(xx),v)
    except:
      xi = np.matmul(np.linalg.pinv(xx),v)
    xtimesxi = np.matmul(X,xi).flatten()
    absxtimesxi = np.absolute(xtimesxi)
    sortedabs = np.sort(absxtimesxi)[::-1]
    absxtimexi_list.append(sortedabs)
  absxtimexi_list = np.array(absxtimexi_list)
  totsums = np.sum(absxtimexi_list, axis = 1)
  for i in range(1,len(list(c))):
    partsums = np.sum(absxtimexi_list[:,:i], axis = 1)
    cc = np.true_divide(partsums,totsums)
    mcc = max(cc)
    if mcc > 0.5:
      return i

In [19]:
def X_lasso(name, lbd=1):
  X,y = read_data(name)
  n, p = X.shape
  XX = - lbd * np.eye(p)
  XX[0][0] = 0
  X1 = np.concatenate((X, XX))
  return X1

In [20]:
def X_lasso_C(X, lbd=1):
  n, p = X.shape
  XX = - lbd * np.eye(p)
  X1 = np.concatenate((X, XX))
  return X1

In [21]:
def select_col(X, beta_l):
  l = []
  for b in range(len(beta_l)):
    if abs(beta_l[b] - 0) > 0.000001:
      l.append(b)
  xx = np.take(X, l, 1)
  return breakdown(xx)

In [22]:
def select_col_uncenter(X0, beta_l):
  l = [0]
  for b in range(len(beta_l)):
    if abs(beta_l[b] - 0) > 0.000001:
      l.append(b+1)
  xx = np.take(X0, l, 1)
  return breakdown(xx)

In [23]:
def print_sheet(dataset, method, end, step):
  array = []
  array.append(['lambda',	'beta',	'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'])
  if method == 'a':
    X, y = read_data(dataset)
  if method == 'b':
    X, y = sd_scale(dataset)
  if method == 'c':
    X, y = mad_scale(dataset)
  cb = breakdown(X)
  for i in range(end+1):
    lbd = i*step
    obj, beta_l = LAD_LASSO(X, y, lbd)
    cbl = breakdown(X_lasso(dataset, lbd))
    cbs = select_col(X, beta_l)
    array.append([round(lbd,1), beta_l, obj, cb, cbl, cbs])
  print(array)
  df = pd.DataFrame(array)
  filepath = '/content/drive/My Drive/Bernice Research/Nov 10/%s.xlsx' % dataset
  if Path(filepath).is_file():
    book = openpyxl.load_workbook(filepath)
    writer = pd.ExcelWriter(filepath, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, index=False, header=False, sheet_name = method)
    writer.save()
  else:
    df.to_excel(filepath, index=False, header=False, sheet_name = method)

In [24]:
def print_sheet_center(dataset, method, end, step):
  array = []
  array.append(['lambda',	'beta',	'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'])
  if method == 'd':
    X, y = mean_center(dataset)
    X0, y0 = read_data(dataset)
  if method == 'e':
    X, y = median_center(dataset)
    X0, y0 = read_data(dataset)
  if method == 'f':
    X, y = mean_center_scale(dataset)
    X0, y0 = sd_scale(dataset)
  if method == 'g':
    X, y = median_center_scale(dataset)
    X0, y0 = mad_scale(dataset)
  cb = breakdown(X)
  for i in range(end+1):
    lbd = i*step
    obj, beta_l = LAD_LASSO_C(X, y, lbd)
    cbl = breakdown(X_lasso_C(X, lbd))
    cbs = select_col(X, beta_l)
    cbsu = select_col_uncenter(X0, beta_l)
    if method == 'd' or method == 'f':
      beta_l.insert(0,round(np.mean(y0),6))
    if method == 'e' or method == 'g':
      beta_l.insert(0,round(np.median(y0),6))
    array.append([round(lbd,1), beta_l, obj, cb, cbl, cbsu])
  print(array)
  df = pd.DataFrame(array)
  filepath = '/content/drive/My Drive/Bernice Research/Nov 10/%s.xlsx' % dataset
  if Path(filepath).is_file():
    book = openpyxl.load_workbook(filepath)
    writer = pd.ExcelWriter(filepath, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, index=False, header=False, sheet_name = method)
    writer.save()
  else:
    df.to_excel(filepath, index=False, header=False, sheet_name = method)

In [31]:
i = 50
dataset = 'salinity'
X, y = read_data(dataset)
obj, beta_l = LAD_LASSO(X, y, i)
print(beta_l)

[5.792593, 0.481481, 0.0, 0.0]


In [ ]:
i = 50
dataset = 'aircraft'
X, y = read_data(dataset)
obj, beta_l = LAD_LASSO(X, y, i)
print(beta_l)

[-13.46374, 0.0, 0.0, 0.002758, -0.000955]


In [45]:
i = 20
dataset = 'salinity'
X, y = sd_scale(dataset)
obj, beta_l = LAD_LASSO(X, y, i)
print(beta_l)

[10.8, 0.0, 0.0, 0.0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [51]:
i = 24
dataset = 'salinity'
X, y = mad_scale(dataset)
obj, beta_l = LAD_LASSO(X, y, i)
print(beta_l)

[10.8, 0.0, 0.0, 0.0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [63]:
i = 57
dataset = 'salinity'
X, y = median_center(dataset)
obj, beta_l = LAD_LASSO_C(X, y, i)
print(beta_l)

[0.122807, 0.0, 0.0]


In [77]:
i = 19
dataset = 'salinity'
X, y = mean_center_scale(dataset)
obj, beta_l = LAD_LASSO_C(X, y, i)
print(beta_l)

[0.908995, 0.0, 0.0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [32]:
print_sheet('salinity', 'a', end=20, step=3)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [14.212097, 0.739684, -0.111747, -0.457656], 24.416163, 4, 4, 4], [3, [12.887341, 0.721444, -0.018468, -0.404982], 28.073166, 4, 4, 4], [6, [12.955865, 0.720236, 0.0, -0.408271], 31.486949, 4, 5, 4], [9, [12.955865, 0.720236, 0.0, -0.408271], 34.872471, 4, 4, 4], [12, [12.929997, 0.71308, 0.0, -0.405993], 38.24543, 4, 4, 4], [15, [12.929997, 0.71308, 0.0, -0.405993], 41.602652, 4, 4, 4], [18, [11.094048, 0.741692, 0.0, -0.337113], 44.901918, 4, 3, 4], [21, [11.094048, 0.741692, 0.0, -0.337113], 48.138332, 4, 3, 4], [24, [7.805414, 0.729168, 0.0, -0.202704], 51.084108, 4, 2, 4], [27, [4.801362, 0.635543, 0.0, -0.027241], 53.402418, 4, 2, 4], [30, [4.801362, 0.635543, 0.0, -0.027241], 55.390768, 4, 2, 4], [33, [4.76056, 0.618195, 0.0, -0.020626], 57.331264, 4, 2, 4], [36, [4.76056, 0.618195, 0.0, -0.020626], 59.247727, 4, 2, 4], [39, [4.76056, 0.618195, 0.0, -0.020626], 61.16419, 4, 2, 4], [

In [46]:
print_sheet('salinity', 'b', end=20, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [14.212097, 2.250534, -0.196977, -1.260346], 24.416163, 4, 4, 4], [1, [12.887341, 2.195037, -0.032553, -1.115285], 27.981362, 4, 4, 4], [2, [12.887341, 2.195037, -0.032553, -1.115285], 31.324238, 4, 4, 4], [3, [12.955865, 2.191363, 0.0, -1.124344], 34.663027, 4, 4, 4], [4, [12.929997, 2.169591, 0.0, -1.118072], 37.967196, 4, 5, 4], [5, [12.929997, 2.169591, 0.0, -1.118072], 41.254858, 4, 5, 4], [6, [12.929997, 2.169591, 0.0, -1.118072], 44.542521, 4, 5, 4], [7, [11.094048, 2.256643, 0.0, -0.92838], 47.778598, 4, 4, 4], [8, [7.805414, 2.218538, 0.0, -0.558229], 50.933327, 4, 4, 4], [9, [5.531989, 1.915432, 0.0, -0.157066], 53.547941, 4, 4, 4], [10, [4.801362, 1.933678, 0.0, -0.075018], 55.59423, 4, 4, 4], [11, [4.76056, 1.880897, 0.0, -0.056802], 57.564857, 4, 4, 4], [12, [5.095105, 1.847742, 0.0, -0.075018], 59.49834, 4, 4, 4], [13, [5.095105, 1.847742, 0.0, -0.075018], 61.4211, 4, 4, 4], 

In [52]:
print_sheet('salinity', 'c', end=24, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [14.212097, 1.857135, -0.175602, -0.868549], 24.416163, 4, 4, 4], [1, [12.783269, 1.86257, -0.099825, -0.763898], 27.230633, 4, 4, 4], [2, [12.887341, 1.811339, -0.029021, -0.768583], 29.856372, 4, 4, 4], [3, [12.955865, 1.808307, 0.0, -0.774825], 32.465304, 4, 4, 4], [4, [12.955865, 1.808307, 0.0, -0.774825], 35.048437, 4, 5, 4], [5, [12.929997, 1.790341, 0.0, -0.770503], 37.620765, 4, 5, 4], [6, [12.929997, 1.790341, 0.0, -0.770503], 40.181609, 4, 5, 4], [7, [12.929997, 1.790341, 0.0, -0.770503], 42.742453, 4, 4, 4], [8, [12.929997, 1.790341, 0.0, -0.770503], 45.303297, 4, 4, 4], [9, [12.929997, 1.790341, 0.0, -0.770503], 47.864141, 4, 4, 4], [10, [12.929997, 1.790341, 0.0, -0.770503], 50.424986, 4, 4, 4], [11, [12.929997, 1.790341, 0.0, -0.770503], 52.98583, 4, 4, 4], [12, [7.215757, 1.642812, 0.0, -0.259052], 55.078952, 4, 4, 4], [13, [5.531989, 1.58061, 0.0, -0.10824], 56.850502, 4, 4

In [60]:
print_sheet_center('salinity', 'd', end=19, step=3)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [10.553571, 0.7033, -0.096756, -0.546208], 24.562003, 5, 5, 4], [3, [10.553571, 0.715505, -0.033984, -0.47406], 28.370173, 5, 6, 4], [6, [10.553571, 0.722113, 0.0, -0.435001], 31.964188, 5, 5, 4], [9, [10.553571, 0.726391, 0.0, -0.378788], 35.285793, 5, 5, 4], [12, [10.553571, 0.726391, 0.0, -0.378788], 38.601328, 5, 4, 4], [15, [10.553571, 0.726391, 0.0, -0.378788], 41.916864, 5, 4, 4], [18, [10.553571, 0.726391, 0.0, -0.378788], 45.232399, 5, 3, 4], [21, [10.553571, 0.730543, 0.0, -0.217465], 48.272002, 5, 3, 4], [24, [10.553571, 0.720163, 0.0, -0.194739], 51.102414, 5, 2, 4], [27, [10.553571, 0.700928, 0.0, -0.152626], 53.740004, 5, 2, 4], [30, [10.553571, 0.635766, 0.0, -0.00996], 55.694929, 5, 2, 4], [33, [10.553571, 0.635766, 0.0, -0.00996], 57.632106, 5, 2, 4], [36, [10.553571, 0.626132, 0.0, -0.009893], 59.55085, 5, 2, 4], [39, [10.553571, 0.626132, 0.0, -0.009893], 61.458924, 5, 2

In [64]:
print_sheet_center('salinity', 'e', end=19, step=3)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [11.15, 0.708385, -0.052259, -0.504666], 24.669745, 4, 4, 4], [3, [11.15, 0.736826, 0.0, -0.395397], 28.305452, 4, 5, 4], [6, [11.15, 0.740106, 0.0, -0.38035], 31.668465, 4, 5, 4], [9, [11.15, 0.740106, 0.0, -0.38035], 35.029834, 4, 4, 4], [12, [11.15, 0.705999, 0.0, -0.40374], 38.382118, 4, 4, 4], [15, [11.15, 0.705999, 0.0, -0.40374], 41.711335, 4, 4, 4], [18, [11.15, 0.705999, 0.0, -0.40374], 45.040552, 4, 3, 4], [21, [11.15, 0.737948, 0.0, -0.296936], 48.183218, 4, 3, 4], [24, [11.15, 0.737948, 0.0, -0.296936], 51.287869, 4, 3, 4], [27, [11.15, 0.674747, 0.0, -0.20202], 53.946667, 4, 2, 4], [30, [11.15, 0.584876, 0.0, -0.048096], 56.037755, 4, 2, 4], [33, [11.15, 0.584876, 0.0, -0.048096], 57.936672, 4, 2, 4], [36, [11.15, 0.572453, 0.0, -0.048392], 59.801282, 4, 2, 4], [39, [11.15, 0.572453, 0.0, -0.048392], 61.663818, 4, 2, 4], [42, [11.15, 0.562529, 0.0, -0.033489], 63.515759, 4, 2,

In [78]:
print_sheet_center('salinity', 'f', end=19, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [10.553571, 2.139833, -0.170552, -1.50421], 24.562003, 5, 5, 4], [1, [10.553571, 2.166815, -0.059211, -1.31554], 28.241585, 5, 6, 4], [2, [10.553571, 2.184455, -0.037598, -1.265469], 31.781262, 5, 5, 4], [3, [10.553571, 2.210088, 0.0, -1.04315], 35.098901, 5, 5, 4], [4, [10.553571, 2.210088, 0.0, -1.04315], 38.352139, 5, 5, 4], [5, [10.553571, 2.210088, 0.0, -1.04315], 41.605377, 5, 4, 4], [6, [10.553571, 2.210088, 0.0, -1.04315], 44.858615, 5, 4, 4], [7, [10.553571, 2.197937, 0.0, -0.718105], 48.078597, 5, 3, 4], [8, [10.553571, 2.222723, 0.0, -0.59888], 50.936654, 5, 3, 4], [9, [10.553571, 2.132617, 0.0, -0.420318], 53.670469, 5, 3, 4], [10, [10.553571, 1.934357, 0.0, -0.027429], 55.941011, 5, 3, 4], [11, [10.553571, 1.934357, 0.0, -0.027429], 57.902796, 5, 2, 4], [12, [10.553571, 1.905045, 0.0, -0.027244], 59.841426, 5, 2, 4], [13, [10.553571, 1.905045, 0.0, -0.027244], 61.773716, 5, 2,

In [79]:
print_sheet_center('salinity', 'g', end=23, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [11.15, 1.778552, -0.082121, -0.957766], 24.669745, 4, 4, 4], [1, [11.15, 1.805221, -0.051452, -0.880319], 27.429865, 4, 5, 4], [2, [11.15, 1.858195, 0.0, -0.721837], 30.105791, 4, 5, 4], [3, [11.15, 1.772563, 0.0, -0.766226], 32.681615, 4, 5, 4], [4, [11.15, 1.772563, 0.0, -0.766226], 35.220403, 4, 5, 4], [5, [11.15, 1.772563, 0.0, -0.766226], 37.759192, 4, 4, 4], [6, [11.15, 1.772563, 0.0, -0.766226], 40.297981, 4, 4, 4], [7, [11.15, 1.772563, 0.0, -0.766226], 42.836769, 4, 4, 4], [8, [11.15, 1.772563, 0.0, -0.766226], 45.375558, 4, 4, 4], [9, [11.15, 1.772563, 0.0, -0.766226], 47.914347, 4, 3, 4], [10, [11.15, 1.772563, 0.0, -0.766226], 50.453135, 4, 3, 4], [11, [11.15, 1.772563, 0.0, -0.766226], 52.991924, 4, 3, 4], [12, [11.15, 1.694098, 0.0, -0.383398], 55.203896, 4, 3, 4], [13, [11.15, 1.501095, 0.0, -0.1643], 57.269341, 4, 3, 4], [14, [11.15, 1.437265, 0.0, -0.09184], 58.858333, 4,

In [ ]:
print_sheet('aircraft', 'a', end=20, step=1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [1.951096, -3.04663, 1.473594, 0.00223, -0.000965], 119.717938, 2, 2, 2], [1, [-0.890917, -2.695816, 1.524527, 0.002344, -0.001003], 124.000091, 2, 2, 2], [2, [-1.088514, -2.560406, 1.480391, 0.002333, -0.000992], 128.173639, 2, 2, 2], [3, [-1.088514, -2.560406, 1.480391, 0.002333, -0.000992], 132.217759, 2, 2, 2], [4, [-4.080457, -2.114097, 1.32641, 0.002379, -0.000883], 135.979478, 2, 2, 2], [5, [-9.247434, -0.90719, 1.197744, 0.002358, -0.000799], 138.599723, 2, 2, 2], [6, [-9.247434, -0.90719, 1.197744, 0.002358, -0.000799], 140.707815, 2, 2, 2], [7, [-9.247434, -0.90719, 1.197744, 0.002358, -0.000799], 142.815906, 2, 2, 2], [8, [-9.247434, -0.90719, 1.197744, 0.002358, -0.000799], 144.923998, 2, 2, 2], [9, [-13.46374, 0.0, 0.0, 0.002758, -0.000955], 145.859706, 2, 2, 4], [10, [-13.46374, 0.0, 0.0, 0.002758, -0.000955], 145.86342, 2, 2, 4], [11, [-13.46374, 0.0, 0.0, 0.002758, -0.00095

In [ ]:
print_sheet('aircraft', 'b', end=17, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [1.951096, -4.205169, 2.603973, 20.811713, -9.847212], 119.717938, 2, 2, 2], [1, [-1.207813, -3.370529, 2.44748, 20.807872, -9.035522], 156.845672, 2, 2, 2], [2, [6.599943, -2.910578, 0.699039, 10.011372, 0.0], 174.135966, 2, 2, 2], [3, [8.491041, -3.172244, 0.715562, 9.198054, 0.0], 187.678169, 2, 2, 2], [4, [11.149037, -2.937397, 0.707298, 6.866584, 0.0], 199.892606, 2, 2, 2], [5, [10.839606, -2.724761, 0.761244, 6.414041, 0.0], 209.802448, 2, 2, 2], [6, [10.216007, -2.508156, 0.731902, 6.490634, 0.0], 219.587386, 2, 2, 2], [7, [10.216007, -2.508156, 0.731902, 6.490634, 0.0], 229.318077, 2, 2, 2], [8, [10.216007, -2.508156, 0.731902, 6.490634, 0.0], 239.048768, 2, 2, 2], [9, [10.216007, -2.508156, 0.731902, 6.490634, 0.0], 248.779459, 2, 2, 2], [10, [11.556834, -2.027183, 0.0, 4.952216, 0.0], 256.477284, 2, 2, 4], [11, [11.631483, -1.817759, 0.0, 4.720493, 0.0], 263.136, 2, 2, 4], [12, [

In [ ]:
print_sheet('aircraft', 'c', end=20, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [1.951096, -3.470509, 1.646581, 15.506338, -7.914083], 119.717938, 2, 2, 2], [1, [1.911314, -3.445626, 1.634587, 15.448009, -7.839712], 148.197183, 2, 2, 2], [2, [6.599943, -2.402089, 0.442026, 7.459248, 0.0], 167.500713, 2, 2, 2], [3, [6.599943, -2.402089, 0.442026, 7.459248, 0.0], 177.804075, 2, 2, 2], [4, [6.599943, -2.402089, 0.442026, 7.459248, 0.0], 188.107438, 2, 2, 2], [5, [11.149037, -2.424223, 0.447248, 5.116137, 0.0], 197.78553, 2, 2, 2], [6, [10.839606, -2.248735, 0.48136, 4.778957, 0.0], 205.356533, 2, 2, 2], [7, [10.216007, -2.069971, 0.462806, 4.836025, 0.0], 212.784856, 2, 2, 2], [8, [10.216007, -2.069971, 0.462806, 4.836025, 0.0], 220.153658, 2, 2, 2], [9, [10.216007, -2.069971, 0.462806, 4.836025, 0.0], 227.52246, 2, 2, 2], [10, [10.216007, -2.069971, 0.462806, 4.836025, 0.0], 234.891262, 2, 2, 2], [11, [10.216007, -2.069971, 0.462806, 4.836025, 0.0], 242.260065, 2, 2, 2]

In [ ]:
print_sheet_center('aircraft', 'd', end=20, step=1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [20.326957, -2.140201, 1.334875, 0.002382, -0.000885], 122.075249, 3, 3, 2], [1, [20.326957, -2.140201, 1.334875, 0.002382, -0.000885], 125.553592, 3, 3, 2], [2, [20.326957, -2.140201, 1.334875, 0.002382, -0.000885], 129.031935, 3, 3, 2], [3, [20.326957, -2.140201, 1.334875, 0.002382, -0.000885], 132.510278, 3, 3, 2], [4, [20.326957, -2.125587, 1.326795, 0.002373, -0.000879], 135.981984, 3, 3, 2], [5, [20.326957, -0.988404, 1.21467, 0.002332, -0.000811], 138.73355, 3, 3, 2], [6, [20.326957, -0.988404, 1.21467, 0.002332, -0.000811], 140.939768, 3, 2, 2], [7, [20.326957, -0.808224, 1.181825, 0.002321, -0.00076], 142.974931, 3, 2, 2], [8, [20.326957, -0.808224, 1.181825, 0.002321, -0.00076], 144.968061, 3, 2, 2], [9, [20.326957, 0.0, 0.0, 0.002584, -0.000804], 145.955379, 3, 2, 4], [10, [20.326957, 0.0, 0.0, 0.002584, -0.000804], 145.958767, 3, 2, 4], [11, [20.326957, 0.0, 0.0, 0.002584, -0.0

In [ ]:
print_sheet_center('aircraft', 'e', end=20, step=1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [13.59, -1.962976, 1.277402, 0.002364, -0.000873], 123.558308, 2, 2, 2], [1, [13.59, -1.962976, 1.277402, 0.002364, -0.000873], 126.801923, 2, 2, 2], [2, [13.59, -1.962976, 1.277402, 0.002364, -0.000873], 130.045537, 2, 3, 2], [3, [13.59, -1.640659, 1.283915, 0.002425, -0.000898], 133.182379, 2, 2, 2], [4, [13.59, -1.640659, 1.283915, 0.002425, -0.000898], 136.110275, 2, 2, 2], [5, [13.59, -0.988049, 1.21075, 0.00239, -0.000831], 138.620795, 2, 2, 2], [6, [13.59, -0.988049, 1.21075, 0.00239, -0.000831], 140.822815, 2, 2, 2], [7, [13.59, -0.988049, 1.21075, 0.00239, -0.000831], 143.024834, 2, 2, 2], [8, [13.59, -0.405838, 1.145477, 0.002358, -0.000771], 145.016133, 2, 2, 2], [9, [13.59, 0.0, 0.0, 0.002707, -0.000911], 145.887893, 2, 2, 4], [10, [13.59, 0.0, 0.0, 0.002707, -0.000911], 145.89151, 2, 2, 4], [11, [13.59, 0.0, 0.0, 0.002707, -0.000911], 145.895128, 2, 2, 4], [12, [13.59, 0.0, 0.

In [ ]:
print_sheet_center('aircraft', 'f', end=17, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [20.326957, -2.954053, 2.358846, 22.221732, -9.031295], 122.075249, 3, 3, 2], [1, [20.326957, -2.933882, 2.344567, 22.146108, -8.970257], 158.55426, 3, 4, 2], [2, [20.326957, -2.758827, 0.537204, 11.717976, 0.0], 183.076621, 3, 4, 2], [3, [20.326957, -2.53838, 0.247175, 9.597357, 0.0], 196.025424, 3, 3, 2], [4, [20.326957, -2.53838, 0.247175, 9.597357, 0.0], 208.408336, 3, 2, 2], [5, [20.326957, -2.460529, 0.241262, 9.59534, 0.0], 220.724509, 3, 2, 2], [6, [20.326957, -2.460529, 0.241262, 9.59534, 0.0], 233.02164, 3, 2, 2], [7, [20.326957, -2.460529, 0.241262, 9.59534, 0.0], 245.31877, 3, 2, 2], [8, [20.326957, -2.013273, 0.207291, 9.58375, 0.0], 257.172257, 3, 2, 2], [9, [20.326957, -1.922875, 0.200425, 9.581408, 0.0], 268.94476, 3, 2, 2], [10, [20.326957, -2.057692, 0.0, 9.529321, 0.0], 280.561815, 3, 2, 4], [11, [20.326957, -2.057692, 0.0, 9.529321, 0.0], 292.148829, 3, 2, 4], [12, [20.

In [ ]:
print_sheet_center('aircraft', 'g', end=20, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [13.59, -2.236086, 1.427358, 16.433286, -7.161272], 123.558308, 2, 2, 2], [1, [13.59, -2.184373, 1.253654, 14.151657, -5.621869], 150.000381, 2, 3, 2], [2, [13.59, -1.97878, 0.712601, 8.168568, -0.27726], 169.940028, 2, 3, 2], [3, [13.59, -1.859184, 0.644143, 6.799978, 0.0], 180.432888, 2, 3, 2], [4, [13.59, -1.859184, 0.644143, 6.799978, 0.0], 189.736192, 2, 3, 2], [5, [13.59, -2.004377, 0.598192, 5.159798, 0.0], 198.694556, 2, 2, 2], [6, [13.59, -1.865711, 0.592923, 5.223495, 0.0], 206.391618, 2, 2, 2], [7, [13.59, -1.865711, 0.592923, 5.223495, 0.0], 214.073747, 2, 2, 2], [8, [13.59, -1.865711, 0.592923, 5.223495, 0.0], 221.755876, 2, 2, 2], [9, [13.59, -1.865711, 0.592923, 5.223495, 0.0], 229.438005, 2, 2, 2], [10, [13.59, -1.869129, 0.566098, 4.397886, 0.0], 236.439168, 2, 2, 2], [11, [13.59, -1.869129, 0.566098, 4.397886, 0.0], 243.272281, 2, 2, 2], [12, [13.59, -1.87056, 0.554871, 4

In [ ]:
print_sheet('wood', 'a', end=10, step=0.1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0.0, [0.424093, 0.406509, -1.564256, -0.324531, 0.119953, 0.18077], 0.300761, 3, 3, 3], [0.1, [0.495348, 0.183979, -0.93884, -0.076487, 0.184001, -0.036589], 0.468741, 3, 4, 3], [0.2, [0.375844, 0.138116, 0.0, 0.0, 0.252371, -0.093703], 0.559363, 3, 2, 3], [0.3, [0.375844, 0.138116, 0.0, 0.0, 0.252371, -0.093703], 0.607782, 3, 2, 3], [0.4, [0.331047, 0.155566, 0.0, 0.0, 0.162376, 0.0], 0.650861, 3, 1, 4], [0.5, [0.340695, 0.158929, 0.0, 0.0, 0.137557, 0.0], 0.681748, 3, 1, 4], [0.6, [0.340695, 0.158929, 0.0, 0.0, 0.137557, 0.0], 0.711397, 3, 1, 4], [0.7, [0.476039, 0.061111, 0.0, 0.0, 0.0, 0.0], 0.718639, 3, 1, 5], [0.8, [0.494053, 0.026667, 0.0, 0.0, 0.0, 0.0], 0.7232, 3, 1, 5], [0.9, [0.494053, 0.026667, 0.0, 0.0, 0.0, 0.0], 0.725867, 3, 1, 5], [1.0, [0.508, 0.0, 0.0, 0.0, 0.0, 0.0], 0.726, 3, 1, None]]


In [ ]:
print_sheet('wood', 'b', end=11, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [0.424093, 0.03608, -0.033708, -0.020463, 0.007345, 0.009251], 0.300761, 3, 3, 3], [1, [0.504154, 0.019239, -0.023693, -0.009424, 0.010982, 0.0], 0.372503, 3, 1, 3], [2, [0.479437, 0.018322, -0.022422, -0.005294, 0.009732, 0.0], 0.434872, 3, 1, 3], [3, [0.496981, 0.016136, -0.020267, -0.004577, 0.011182, -0.001951], 0.490259, 3, 1, 3], [4, [0.461789, 0.017667, -0.021515, -0.002346, 0.008839, 0.0], 0.541945, 3, 1, 3], [5, [0.445352, 0.014382, -0.015731, 0.0, 0.007235, 0.0], 0.586873, 3, 1, 3], [6, [0.435525, 0.012123, -0.014585, 0.0, 0.009042, 0.0], 0.622789, 3, 1, 3], [7, [0.43728, 0.006847, -0.013331, 0.0, 0.011209, 0.0], 0.654956, 3, 1, 3], [8, [0.43728, 0.006847, -0.013331, 0.0, 0.011209, 0.0], 0.686343, 3, 1, 3], [9, [0.43728, 0.006847, -0.013331, 0.0, 0.011209, 0.0], 0.71773, 3, 1, 3], [10, [0.492308, 0.0, 0.0, 0.0, 0.00157, 0.0], 0.72493, 3, 1, 5], [11, [0.508, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
print_sheet('wood', 'c', end=12, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [0.424093, 0.030061, -0.028673, -0.015886, 0.00613, 0.00733], 0.300761, 3, 3, 3], [1, [0.504154, 0.01603, -0.020154, -0.007316, 0.009166, 0.0], 0.36183, 3, 1, 3], [2, [0.504154, 0.01603, -0.020154, -0.007316, 0.009166, 0.0], 0.414496, 3, 1, 3], [3, [0.479437, 0.015266, -0.019073, -0.00411, 0.008122, 0.0], 0.463044, 3, 1, 3], [4, [0.496981, 0.013445, -0.01724, -0.003553, 0.009332, -0.001546], 0.508382, 3, 1, 3], [5, [0.461789, 0.01472, -0.018301, -0.001821, 0.007377, 0.0], 0.551574, 3, 1, 3], [6, [0.445352, 0.011983, -0.013382, 0.0, 0.006038, 0.0], 0.588548, 3, 1, 3], [7, [0.435525, 0.0101, -0.012406, 0.0, 0.007547, 0.0], 0.618663, 3, 1, 3], [8, [0.43728, 0.005705, -0.01134, 0.0, 0.009355, 0.0], 0.646443, 3, 1, 3], [9, [0.43728, 0.005705, -0.01134, 0.0, 0.009355, 0.0], 0.672843, 3, 1, 3], [10, [0.43728, 0.005705, -0.01134, 0.0, 0.009355, 0.0], 0.699242, 3, 1, 3], [11, [0.494053, 0.001972, 0

In [ ]:
print_sheet_center('wood', 'd', end=7, step=0.1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0.0, [0.501, 0.384765, -1.478523, -0.326942, 0.136626, 0.100645], 0.302518, 3, 3, 3], [0.1, [0.501, 0.239954, -0.732916, -0.102121, 0.124482, 0.0], 0.488961, 3, 4, 3], [0.2, [0.501, 0.274348, 0.0, 0.0, 0.15329, -0.06731], 0.575778, 3, 2, 3], [0.3, [0.501, 0.230012, 0.0, 0.0, 0.138938, -0.024527], 0.620559, 3, 2, 3], [0.4, [0.501, 0.199639, 0.0, 0.0, 0.136162, 0.0], 0.655127, 3, 1, 4], [0.5, [0.501, 0.145558, 0.0, 0.0, 0.152448, 0.0], 0.685188, 3, 1, 4], [0.6, [0.501, 0.137511, 0.0, 0.0, 0.107437, 0.0], 0.711644, 3, 1, 4], [0.7, [0.501, 0.0, 0.0, 0.0, 0.017007, 0.0], 0.736007, 3, 1, 5]]


In [ ]:
print_sheet_center('wood', 'e', end=7, step=0.1)

[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0.0, [0.507, 0.342455, -1.435409, -0.30235, 0.095228, -0.042664], 0.325824, 3, 3, 3], [0.1, [0.507, 0.201008, -0.55722, -0.063503, 0.245438, 0.0], 0.518114, 3, 4, 3], [0.2, [0.507, 0.119397, 0.0, 0.0, 0.39972, -0.07143], 0.605692, 3, 2, 3], [0.3, [0.507, 0.101471, 0.0, 0.0, 0.121878, 0.0], 0.654963, 3, 2, 4], [0.4, [0.507, 0.097705, 0.0, 0.0, 0.104981, 0.0], 0.677027, 3, 1, 4], [0.5, [0.507, 0.097705, 0.0, 0.0, 0.104981, 0.0], 0.697295, 3, 1, 4], [0.6, [0.507, 0.080327, 0.0, 0.0, 0.027022, 0.0], 0.712306, 3, 1, 4], [0.7, [0.507, 0.074303, 0.0, 0.0, 0.0, 0.0], 0.722663, 3, 1, 5]]


In [ ]:
print_sheet_center('wood', 'f', end=11, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [0.501, 0.03415, -0.03186, -0.020615, 0.008365, 0.005151], 0.302518, 3, 3, 3], [1, [0.501, 0.030434, -0.029278, -0.018872, 0.008531, 0.0], 0.39442, 3, 4, 3], [2, [0.501, 0.020443, -0.020022, -0.009891, 0.007214, 0.0], 0.461495, 3, 4, 3], [3, [0.501, 0.018536, -0.017644, -0.00439, 0.007113, 0.0], 0.516092, 3, 4, 3], [4, [0.501, 0.018536, -0.017644, -0.00439, 0.007113, 0.0], 0.563775, 3, 3, 3], [5, [0.501, 0.017015, -0.015747, 0.0, 0.007033, 0.0], 0.604996, 3, 3, 3], [6, [0.501, 0.012355, -0.012019, 0.0, 0.00658, 0.0], 0.638774, 3, 3, 3], [7, [0.501, 0.012266, -0.012037, 0.0, 0.006237, 0.0], 0.669554, 3, 2, 3], [8, [0.501, 0.007427, -0.008256, 0.0, 0.005426, 0.0], 0.698505, 3, 2, 3], [9, [0.501, 0.007427, -0.008256, 0.0, 0.005426, 0.0], 0.719614, 3, 2, 3], [10, [0.501, 0.0, -0.002454, 0.0, 0.00418, 0.0], 0.72906, 3, 2, 4], [11, [0.501, 0.0, 0.0, 0.0, 0.004252, 0.0], 0.734188, 3, 2, 5]]


In [ ]:
print_sheet_center('wood', 'g', end=12, step=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[['lambda', 'beta', 'LP objective', 'CB', 'CB with LASSO constriants', 'Cols uncentered CB'], [0, [0.507, 0.025325, -0.026311, -0.0148, 0.004866, -0.00173], 0.325824, 3, 3, 3], [1, [0.507, 0.025963, -0.023173, -0.017068, 0.004361, 0.0], 0.397509, 3, 4, 3], [2, [0.507, 0.021665, -0.023096, -0.01356, 0.004056, 0.0], 0.464226, 3, 4, 3], [3, [0.507, 0.022982, -0.019867, -0.011313, 0.004201, 0.0], 0.525325, 3, 4, 3], [4, [0.507, 0.018389, -0.014416, 0.0, 0.004137, 0.0], 0.566474, 3, 4, 3], [5, [0.507, 0.01777, -0.013905, 0.0, 0.004015, 0.0], 0.603293, 3, 3, 3], [6, [0.507, 0.008108, -0.017379, 0.0, 0.00292, 0.0], 0.635747, 3, 3, 3], [7, [0.507, 0.007911, -0.0158, 0.0, 0.00278, 0.0], 0.663904, 3, 3, 3], [8, [0.507, 0.007437, -0.004883, 0.0, 0.004566, 0.0], 0.684237, 3, 3, 3], [9, [0.507, 0.006515, -0.004606, 0.0, 0.001789, 0.0], 0.698618, 3, 2, 3], [10, [0.507, 0.006464, -0.004564, 0.0, 0.001779, 0.0], 0.711462, 3, 2, 3], [11, [0.507, 0.00094, 0.0, 0.0, 0.000686, 0.0], 0.723603, 3, 2, 4], [1